# Tugas Pemrograman Text Classification menggunakan ML
## 13521009 - Christophorus Dharma Winata

# 0. Notebook setup

In [53]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword_indo = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christodharma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 1. Memuat dataset

In [54]:
test_preprocess = pd.read_csv('dataset/test_preprocess.tsv', sep='\t', header=None, names=['text', 'label'])
test_preprocess_masked_label = pd.read_csv('dataset/test_preprocess_masked_label.tsv', sep='\t', header=None, names=['text', 'label'])
train_preprocess = pd.read_csv('dataset/train_preprocess.tsv', sep='\t', header=None, names=['text', 'label'])
valid_preprocess = pd.read_csv('dataset/valid_preprocess.tsv', sep='\t', header=None, names=['text', 'label'])

# 2. Preprocessing

In [55]:
data_df = pd.concat([train_preprocess, valid_preprocess])
test_df = test_preprocess

# membuat semua teks menjadi lowercase
data_df['text'] = data_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()

label_encoder = LabelEncoder()
data_df['label'] = label_encoder.fit_transform(data_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

# Split into X (features) and y (labels)
X_train = data_df['text']
y_train = data_df['label']
X_test = test_df['text']
y_test = test_df['label']

# 3. Feature extraction

In [56]:
# Bag of Words feature extraction
vectorizer = CountVectorizer(stop_words=stopword_indo, max_features=1000)
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

/Users/christodharma/Stuff/NLP/Tugas Classification with ML/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [57]:
# TFIDF feature extraction
TFIDF_vectorizer = TfidfVectorizer(stop_words=stopword_indo, max_features=1000)
X_train_tfidif = TFIDF_vectorizer.fit_transform(X_train)
X_test_tfidif = TFIDF_vectorizer.transform(X_test)

# 4. Model training

3 Model yang digunakan:
1. Logistic Regression
2. Naive Bayes
3. Support Vector Machine (SVM)

## Eksperimen dengan CountVectorizer

In [58]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_bow, y_train)
y_pred_logreg = logreg.predict(X_test_bow)

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_bow, y_train)
y_pred_nb = nb.predict(X_test_bow)

# Support Vector Machine
svm = SVC(kernel='linear')
svm.fit(X_train_bow, y_train)
y_pred_svm = svm.predict(X_test_bow)

## Eksperimen dengan TFIDFVectorizer

In [59]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_tfidif, y_train)
y_pred_logreg_tfidf = logreg.predict(X_test_tfidif)

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidif, y_train)
y_pred_nb_tfidf = nb.predict(X_test_tfidif)

# Support Vector Machine
svm = SVC(kernel='linear')
svm.fit(X_train_tfidif, y_train)
y_pred_svm_tfidf = svm.predict(X_test_tfidif)

# 5. Model evaluation

In [60]:
def evaluate_model(y_true, y_pred, printResult = True):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    if (printResult):
        print(f'Akurasi: {acc}\nPrecision: {prec}\nRecall: {rec}\nF1-score: {f1}')
    return acc, prec, rec, f1

## Evaluasi pada model dengan CountVectorizer

In [61]:
print("Hasil Logistic Regression: ")
logreg_results = evaluate_model(y_test, y_pred_logreg)
print("\nHasil Naive Bayes: ")
nb_results = evaluate_model(y_test, y_pred_nb)
print("\nHasil SVM: ")
svm_results = evaluate_model(y_test, y_pred_svm)

Hasil Logistic Regression: 
Akurasi: 0.718
Precision: 0.709477620804852
Recall: 0.6615818821701175
F1-score: 0.6728771705300874

Hasil Naive Bayes: 
Akurasi: 0.574
Precision: 0.5419683249140613
Recall: 0.5310257552904611
F1-score: 0.5178924489739062

Hasil SVM: 
Akurasi: 0.718
Precision: 0.7354401154401154
Recall: 0.668106403400521
F1-score: 0.6846018332616475


## Evaluasi pada model dengan TFIDFVectorizer

In [62]:
print("Hasil Logistic Regression: ")
logreg_results = evaluate_model(y_test, y_pred_logreg_tfidf)
print("\nHasil Naive Bayes: ")
nb_results = evaluate_model(y_test, y_pred_nb_tfidf)
print("\nHasil SVM: ")
svm_results = evaluate_model(y_test, y_pred_svm_tfidf)

Hasil Logistic Regression: 
Akurasi: 0.71
Precision: 0.7193901874119395
Recall: 0.6377519539284244
F1-score: 0.6496414322661943

Hasil Naive Bayes: 
Akurasi: 0.62
Precision: 0.5982483297061302
Recall: 0.5459058914941267
F1-score: 0.5447116736990155

Hasil SVM: 
Akurasi: 0.706
Precision: 0.71865372139978
Recall: 0.6433823529411765
F1-score: 0.6566883384473279
